In [ ]:
import os
import gzip

# Paramètres
input_file = "en.openfoodfacts.org.products.csv.gz"  # Fichier CSV.gz source
output_dir = "output_files1_csv"  # Dossier pour les fichiers de sortie
max_lines_per_file = 10000  # Nombre de lignes maximum par fichier, y compris les en-têtes

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

total_size_bytes = os.path.getsize(input_file)  # Taille totale du fichier
print("Taille totale du fichier (Go):", total_size_bytes / (1024 ** 3))

with gzip.open(input_file, "rt", encoding="utf-8") as infile:
    sample_lines = [next(infile) for _ in range(max_lines_per_file)]
    avg_line_size = sum(len(line.encode("utf-8")) for line in sample_lines) / max_lines_per_file
    total_lines = int(total_size_bytes / avg_line_size)

print(f"Estimation du nombre total de lignes : {total_lines}")

# Lire et diviser le fichier
with gzip.open(input_file, "rt", encoding="utf-8") as infile:
    # Lire les en-têtes
    header = infile.readline()
    
    # Initialisation
    file_idx = 1
    line_count = 0
    outfile = None

    for line in infile:
        # Si un nouveau fichier doit être créé
        if line_count % max_lines_per_file == 0:
            # Fermer l'ancien fichier s'il existe
            if outfile:
                outfile.close()
            
            # Créer un nouveau fichier
            output_file = os.path.join(output_dir, f"part_{file_idx}.csv")
            outfile = open(output_file, "w", encoding="utf-8")
            outfile.write(header)  # Ajouter les en-têtes
            file_idx += 1
        
        # Écrire la ligne dans le fichier actuel
        outfile.write(line)
        line_count += 1
    
    # Fermer le dernier fichier
    if outfile:
        outfile.close()

print(f"Fichiers divisés disponibles dans le dossier : {output_dir}")


: 

## Chargement et comprehension du fichier csv

In [1]:
import pandas as pd

# Chemin du fichier CSV
file_path = '/home/gaelle/Téléchargements/output_files1_csv/part_1.csv'

# Chemin du fichier texte contenant les colonnes principales
columns_file_path = 'important_columns.txt'

# Charger les colonnes principales à partir du fichier texte
try:
    with open(columns_file_path, 'r') as file:
        selected_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes depuis le fichier texte : {e}")
    exit()

# Charger le fichier CSV
try:
    df = pd.read_csv(file_path, delimiter='\t', low_memory=False)
except Exception as e:
    print(f"Erreur lors du chargement du fichier : {e}")
    exit()

# Étape 1 : Garder uniquement les colonnes nécessaires
if all(col in df.columns for col in selected_columns):
    df = df[selected_columns]
else:
    print("Les colonnes sélectionnées ne sont pas toutes présentes dans le fichier.")
    exit()

# Étape 2 : Supprimer les doublons
df = df.drop_duplicates()


# Sauvegarder le fichier nettoyé
cleaned_file_path = 'cleaned_file.csv'
df.to_csv(cleaned_file_path, index=False, encoding='utf-8')

# Aperçu des premières lignes
print(f"Fichier nettoyé sauvegardé dans : {cleaned_file_path}")
print(df.head())

Fichier nettoyé sauvegardé dans : cleaned_file.csv
   code                       product_name     creator  \
0     1  Purée Mix Tropical Harmony + Aloe         inf   
1     2  Matcha organic Japanese green tea     kiliweb   
2     3           Slim Jim snack size mild  prepperapp   
3     4                           Marinara      elcoco   
4     5                         Bio inulin   touchette   

                                                 url          brands  \
0  http://world-en.openfoodfacts.org/product/0000...          Punuts   
1  http://world-en.openfoodfacts.org/product/0000...             Zen   
2  http://world-en.openfoodfacts.org/product/0000...  Conagra Brands   
3  http://world-en.openfoodfacts.org/product/0000...    Newman's own   
4  http://world-en.openfoodfacts.org/product/0000...             EWL   

             categories           origins manufacturing_places  \
0                syrups           Spanien                  NaN   
1  Bebidas instantáneas            

In [ ]:
import csv
import urllib.parse
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS
from rdflib.namespace import OWL, XSD, FOAF, DC

# Chemins des fichiers
csv_file_path = "cleaned_file.csv"  # Remplacez par votre fichier CSV
txt_file_path = "important_columns.txt"  # Fichier texte contenant les colonnes importantes
rdf_file_path = "output_ontology.owl"  # Fichier de sortie OWL

# Espaces de noms pour l'ontologie
SCHEMA = Namespace("http://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
EX = Namespace("http://produitsalimentaires.fr/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")



# Charger la liste des colonnes importantes depuis le fichier texte
try:
    with open(txt_file_path, "r") as file:
        important_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes : {e}")
    exit()

# Liaison des namespaces au graphe
g = Graph()
g.bind("schema", SCHEMA)
g.bind("dbo", DBO)
g.bind("foaf", FOAF)
g.bind("dc", DC)
g.bind("ex", EX)
g.bind("owl", OWL)

# Définition de l'ontologie
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDFS.comment, Literal(
    "Une ontologie"
)))

# Définir les classes principales et sous-classes
g.add((EX.Product, RDF.type, OWL.Class))
g.add((DBO.Product, RDF.type, OWL.Class))

g.add((EX.Brand, RDF.type, OWL.Class))
g.add((DBO.Brand, RDF.type, OWL.Class))

g.add((EX.Category, RDF.type, OWL.Class))
g.add((DBO.Category, RDF.type, OWL.Class))

g.add((EX.Nutrient, RDF.type, OWL.Class))
g.add((EX.Allergen, RDF.type, OWL.Class))
#g.add((EX.Origin, RDF.type, OWL.Class))
#g.add((EX.Addiction, RDF.type, OWL.Class))
g.add((EX.Country, RDF.type, OWL.Class))
g.add((DBO.Country, RDF.type, OWL.Class))

g.add((EX.Creator, RDF.type, OWL.Class))
g.add((EX.ManufacturingPlace, RDF.type, OWL.Class))
g.add((EX.Ingredient, RDF.type, OWL.Class))
g.add((EX.PurchasePlace, RDF.type, OWL.Class))
g.add((EX.City, RDF.type, OWL.Class))
#g.add((EX.Ecoscore, RDF.type, OWL.Class))

# Sous-classes potentielles (exemples)
#g.add((EX.EcoscoreScore, RDF.type, OWL.Class))
#g.add((EX.EcoscoreGrade, RDF.type, OWL.Class))

# Définir les propriétés et leur domaine et plage
g.add((EX.hasBrand, RDF.type, OWL.ObjectProperty))
g.add((EX.hasBrand, RDFS.domain, EX.Product))
g.add((EX.hasBrand, RDFS.range, EX.Brand))

g.add((EX.hasCategory, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCategory, RDFS.domain, EX.Product))
g.add((EX.hasCategory, RDFS.range, EX.Category))

#g.add((EX.hasOrigin, RDF.type, OWL.ObjectProperty))
#g.add((EX.hasOrigin, RDFS.domain, EX.Product))
#g.add((EX.hasOrigin, RDFS.range, EX.Origin))

g.add((EX.containsAllergen, RDF.type, OWL.ObjectProperty))
g.add((EX.containsAllergen, RDFS.domain, EX.Product))
g.add((EX.containsAllergen, RDFS.range, EX.Allergen))

g.add((EX.hasNutrient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasNutrient, RDFS.domain, EX.Product))
g.add((EX.hasNutrient, RDFS.range, EX.Nutrient))

#g.add((EX.hasAddiction, RDF.type, OWL.ObjectProperty))
#g.add((EX.hasAddiction, RDFS.domain, EX.Product))
#g.add((EX.hasAddiction, RDFS.range, EX.Addiction))

g.add((EX.availableInCountry, RDF.type, OWL.ObjectProperty))
g.add((EX.availableInCountry, RDFS.domain, EX.Product))
g.add((EX.availableInCountry, RDFS.range, DBO.Country))

# Nouvelles propriétés pour les entités supplémentaires
g.add((EX.hasCreator, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCreator, RDFS.domain, EX.Product))
g.add((EX.hasCreator, RDFS.range, EX.Creator))

g.add((EX.hasManufacturingPlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasManufacturingPlace, RDFS.domain, EX.Product))
g.add((EX.hasManufacturingPlace, RDFS.range, EX.ManufacturingPlace))

g.add((EX.hasIngredient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasIngredient, RDFS.domain, EX.Product))
g.add((EX.hasIngredient, RDFS.range, EX.Ingredient))



g.add((EX.hasIngredientsText, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasIngredientsText, RDFS.domain, EX.Product))
g.add((EX.hasIngredientsText, RDFS.range, XSD.string))

# Ajouter les propriétés pour quantity et serving_size
g.add((EX.hasQuantity, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasQuantity, RDFS.domain, EX.Product))
g.add((EX.hasQuantity, RDFS.range, XSD.string))

#g.add((EX.hasServingSize, RDF.type, OWL.DatatypeProperty))
#g.add((EX.hasServingSize, RDFS.domain, EX.Product))
#g.add((EX.hasServingSize, RDFS.range, XSD.string))


g.add((EX.hasPurchasePlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasPurchasePlace, RDFS.domain, EX.Product))
g.add((EX.hasPurchasePlace, RDFS.range, EX.PurchasePlace))

g.add((EX.hasCity, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCity, RDFS.domain, EX.Product))
g.add((EX.hasCity, RDFS.range, EX.City))

"""g.add((EX.hasEcoscore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscore, RDFS.domain, EX.Product))
g.add((EX.hasEcoscore, RDFS.range, EX.Ecoscore))

g.add((EX.hasEcoscoreScore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreScore, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreScore, RDFS.range, EX.EcoscoreScore))

#g.add((EX.hasEcoscoreGrade, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreGrade, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreGrade, RDFS.range, EX.EcoscoreGrade))"""

# Fonction pour encoder les URI de manière sécurisée
def encode_uri(uri):
    return urllib.parse.quote(uri, safe=":/#?&=~")

# Lire le fichier CSV et ajouter des instances
try:
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
          
            if all(col in row for col in important_columns):
                product_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/{row['code']}"))

                # Ajouter le type de produit
                
                g.add((product_uri, RDF.type, DBO.Product))
                g.add((product_uri, SCHEMA.name, Literal(row["product_name"])))
                g.add((product_uri, SCHEMA.url, URIRef(row["url"])))
             


                # Ajouter la marque
                if row["brands"]:
                    brand_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/brand/{row['brands']}"))
                    g.add((product_uri, EX.hasBrand, brand_uri))
                    g.add((brand_uri, RDF.type, DBO.Brand))
                    g.add((brand_uri, FOAF.name, Literal(row["brands"], lang="en")))

                # Ajouter des catégories basées sur le champ 'categories'
                if row["categories"]:
                    categories = row["categories"].split(',')  # Séparer les catégories par des virgules
                    for category in categories:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            category_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/category/{category}"))
                            
                            g.add((product_uri, EX.hasCategory, category_uri))
                            g.add((category_uri, RDF.type, EX.Category))
                            g.add((category_uri, FOAF.name, Literal(category, lang="en")))

                # Ajouter des relations d'origine basées sur le champ 'origins'
                #if row["origins"]:
                    #origins = row["origins"].split(',')  # Séparer les valeurs par des virgules
                    #for origin in origins:
                       # origin = origin.strip()
                        #if origin:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                           # origin_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/origin/{origin}"))
                            
                            #g.add((product_uri, EX.hasOrigin, origin_uri))
                            #g.add((origin_uri, RDF.type, EX.Origin))
                            #g.add((origin_uri, SCHEMA.name, Literal(origin)))
                    
                if row["allergens"]:
                    allergens = row["allergens"].split(',')  # Séparer les valeurs par des virgules
                    for allergen in allergens:
                        allergen = allergen.strip()
                        if allergen:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                            allergen_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/allergens/{allergen}"))
                            
                            g.add((product_uri, EX.containsAllergen, allergen_uri))
                            g.add((allergen_uri, RDF.type, EX.Allergen))
                            g.add((allergen_uri, FOAF.name, Literal(allergen, lang="en")))

                # Ajouter des addictions
                #if row["additives"]:
                    #addiction_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/addiction/{row['additives']}"))
                    #print(f"Adding addictives URI: {addiction_uri}")
                    #g.add((product_uri, EX.hasAddiction, addiction_uri))
                    #g.add((addiction_uri, RDF.type, EX.Addiction))
                    #g.add((addiction_uri, SCHEMA.name, Literal(row["additives"])))

                # Ajouter des pays de disponibilité
                #if row["countries"]:
                   # country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/country/{row['countries']}"))
                   # g.add((product_uri, EX.availableInCountry, country_uri))
                    #g.add((country_uri, RDF.type, DBO.Country))
                    #g.add((country_uri, SCHEMA.name, Literal(row["countries"])))
                
                if row["countries"]:
                    countries = row["countries"].split(',')  # Séparer les catégories par des virgules
                    for category in countries:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/countries/{category}"))
                            g.add((product_uri, EX.availableInCountry, country_uri))
                            g.add((category_uri, RDF.type, DBO.Country))
                            g.add((category_uri, FOAF.name, Literal(category, lang="en")))
                
                # Ajouter des nutriments
                nutrients = [
                    ("energy_100g", "Energy"),
                    ("fat_100g", "Fat"),
                    ("saturated-fat_100g", "Saturated Fat"),
                    ("carbohydrates_100g", "Carbohydrates"),
                    ("sugars_100g", "Sugars"),
                    ("fiber_100g", "Fiber"),
                    ("proteins_100g", "Proteins"),
                    ("salt_100g", "Salt"),
                    ("carbon-footprint_100g", "carbon footprint")
                ]
                for nutrient, label in nutrients:
                    if row[nutrient]:
                        nutrient_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/nutrient/{label}"))
                        g.add((product_uri, EX.hasNutrient, nutrient_uri))
                        g.add((nutrient_uri, RDF.type, EX.Nutrient))
                        g.add((nutrient_uri, SCHEMA.name, Literal(label)))
                        g.add((nutrient_uri, SCHEMA.value, Literal(row[nutrient], datatype=XSD.float)))
                        
                # Ajouter des ingrédients
                if row["ingredients_text"]:
                    g.add((product_uri, EX.hasIngredientsText, Literal(row["ingredients_text"])))

                # Ajouter les créateurs
                if row["creator"]:
                    creator_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/creator/{row['creator']}"))
                    g.add((product_uri, EX.hasCreator, creator_uri))
                    g.add((creator_uri, RDF.type, EX.Creator))
                    g.add((creator_uri, FOAF.name, Literal(row["creator"], lang="en")))

                # Ajouter les lieux de fabrication
                if row["manufacturing_places"]:
                    manufacturing = row["manufacturing_places"].split(',')  # Séparer les catégories par des virgules
                    for manu in manufacturing:
                        cmanu = manu.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            
                            #Ajouter des instances
                            manufacturing_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/manufacturing_place/{manu}"))
                            
                            # Définir les relations et propriétés
                            g.add((product_uri, EX.hasManufacturingPlace, manufacturing_place_uri))
                            g.add((manufacturing_place_uri, RDF.type, EX.ManufacturingPlace))
                            g.add((manufacturing_place_uri, FOAF.name, Literal(manu, lang="en")))
                
                    

                # Ajouter les lieux d'achat
                if row["purchase_places"]:
                    purchase_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/purchase_place/{row['purchase_places']}"))
                    g.add((product_uri, EX.hasPurchasePlace, purchase_place_uri))
                    g.add((purchase_place_uri, RDF.type, EX.PurchasePlace))
                    g.add((purchase_place_uri, SCHEMA.name, Literal(row["purchase_places"])))

                # Ajouter les villes
                if row["cities"]:
                    city_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/city/{row['cities']}"))
                    g.add((product_uri, EX.hasCity, city_uri))
                    g.add((city_uri, RDF.type, EX.City))
                    g.add((city_uri, FOAF.name, Literal(row["cities"], lang="en")))
                    
                  # Ajouter la quantité (quantity)
                if row["quantity"]:
                    g.add((product_uri, EX.hasQuantity, Literal(row["quantity"])))

                # Ajouter la taille de portion (serving_size)
                #if row["serving_size"]:
                   # g.add((product_uri, EX.hasServingSize, Literal(row["serving_size"])))

                # Ajouter l'écorescore (score et grade)
                

except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    exit()

# Sauvegarder le graphe OWL
g.serialize(destination=rdf_file_path, format="xml")
print(f"Ontologie OWL générée et sauvegardée dans : {rdf_file_path}")


: 

In [2]:
from rdflib import Graph

# Charger le graphe RDF depuis le fichier OWL
file_path = "output_ontology.owl"
g = Graph()
try:
    g.parse(file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()
# Sauvegarder le graphe au format JSON-LD
jsonld_file_path = "output_ontology.jsonld"
g.serialize(destination=jsonld_file_path, format="json-ld", indent=4)
print(f"Graphe RDF converti et sauvegardé au format JSON-LD dans : {jsonld_file_path}")

Ontologie OWL chargée depuis : output_ontology.owl
Graphe RDF converti et sauvegardé au format JSON-LD dans : output_ontology.jsonld


In [3]:
from rdflib import Graph

# Chemins des fichiers
owl_file_path = "output_ontology.owl"  # Chemin vers votre fichier OWL
ttl_file_path = "output_ontology.ttl"  # Chemin du fichier Turtle de sortie

# Charger le fichier OWL
g = Graph()
try:
    g.parse(owl_file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {owl_file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()

# Sauvegarder le graphe en Turtle
try:
    g.serialize(destination=ttl_file_path, format="turtle")
    print(f"Fichier Turtle RDF généré avec succès dans : {ttl_file_path}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier Turtle : {e}")

Ontologie OWL chargée depuis : output_ontology.owl
Fichier Turtle RDF généré avec succès dans : output_ontology.ttl


In [4]:
from rdflib import Graph

# Chemin vers le fichier source (.owl)
input_file_path = "output_ontology.owl"  # Remplacez par le chemin de votre fichier .owl

# Chemin vers le fichier de sortie (.rdf)
output_file_path = "output_ontology.rdf"  # Fichier de sortie au format RDF/XML


# Charger le fichier OWL dans un graphe RDFLib
g = Graph()
try:
    g.parse(input_file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {input_file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()

# Sauvegarder le graphe en rdf
try:
    g.serialize(destination=output_file_path, format="xml")
    print(f"Fichier RDF généré avec succès dans : {output_file_path}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier RDF : {e}")


print(f"Conversion terminée. Le fichier RDF a été enregistré sous : {output_file_path}")

Ontologie OWL chargée depuis : output_ontology.owl
Fichier RDF généré avec succès dans : output_ontology.rdf
Conversion terminée. Le fichier RDF a été enregistré sous : output_ontology.rdf


## Enrichir les Données avec des Liens vers DBpedia et Wikidata

In [1]:
#%pip install SPARQLWrapper
import csv
import json
import urllib.parse
import requests
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS
from rdflib.namespace import OWL, XSD, FOAF, DC
from SPARQLWrapper import SPARQLWrapper, JSON
import logging
import os

# Chemins des fichiers
txt_file_path = "important_columns.txt"  # Fichier texte contenant les colonnes importantes
rdf_file_path = "output_ontology2.rdf"  # Fichier de sortie OWL

# Espaces de noms pour l'ontologie
SCHEMA = Namespace("http://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
EX = Namespace("http://produitsalimentaires.fr/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")

# Charger la liste des colonnes importantes depuis le fichier texte
try:
    with open(txt_file_path, "r") as file:
        important_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes : {e}")
    exit()

# Liaison des namespaces au graphe
g = Graph()
g.bind("schema", SCHEMA)
g.bind("dbo", DBO)
g.bind("foaf", FOAF)
g.bind("dc", DC)
g.bind("ex", EX)
g.bind("owl", OWL)



# Définition de l'ontologie
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDFS.comment, Literal("Une ontologie")))

# Définir les classes principales et sous-classes
g.add((EX.Product, RDF.type, OWL.Class))
g.add((DBO.Product, RDF.type, OWL.Class))

g.add((EX.Brand, RDF.type, OWL.Class))
g.add((DBO.Brand, RDF.type, OWL.Class))

g.add((EX.Category, RDF.type, OWL.Class))
g.add((DBO.Category, RDF.type, OWL.Class))

g.add((EX.Nutrient, RDF.type, OWL.Class))
g.add((EX.Allergen, RDF.type, OWL.Class))

g.add((EX.Creator, RDF.type, OWL.Class))
g.add((EX.ManufacturingPlace, RDF.type, OWL.Class))
g.add((EX.Ingredient, RDF.type, OWL.Class))
g.add((EX.PurchasePlace, RDF.type, OWL.Class))
g.add((EX.City, RDF.type, OWL.Class))


# Définir les propriétés et leur domaine et plage
g.add((EX.hasBrand, RDF.type, OWL.ObjectProperty))
g.add((EX.hasBrand, RDFS.domain, EX.Product))
g.add((EX.hasBrand, RDFS.range, EX.Brand))
g.add((EX.Country, RDF.type, OWL.Class))
g.add((DBO.Country, RDF.type, OWL.Class))

g.add((EX.hasCategory, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCategory, RDFS.domain, EX.Product))
g.add((EX.hasCategory, RDFS.range, EX.Category))


g.add((EX.containsAllergen, RDF.type, OWL.ObjectProperty))
g.add((EX.containsAllergen, RDFS.domain, EX.Product))
g.add((EX.containsAllergen, RDFS.range, EX.Allergen))

g.add((EX.hasNutrient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasNutrient, RDFS.domain, EX.Product))
g.add((EX.hasNutrient, RDFS.range, EX.Nutrient))


g.add((EX.availableInCountry, RDF.type, OWL.ObjectProperty))
g.add((EX.availableInCountry, RDFS.domain, EX.Product))
g.add((EX.availableInCountry, RDFS.range, DBO.Country))

# Nouvelles propriétés pour les entités supplémentaires
g.add((EX.hasCreator, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCreator, RDFS.domain, EX.Product))
g.add((EX.hasCreator, RDFS.range, EX.Creator))

g.add((EX.hasManufacturingPlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasManufacturingPlace, RDFS.domain, EX.Product))
g.add((EX.hasManufacturingPlace, RDFS.range, EX.ManufacturingPlace))

g.add((EX.hasIngredient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasIngredient, RDFS.domain, EX.Product))
g.add((EX.hasIngredient, RDFS.range, EX.Ingredient))

g.add((EX.hasIngredientsText, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasIngredientsText, RDFS.domain, EX.Product))
g.add((EX.hasIngredientsText, RDFS.range, XSD.string))

# Ajouter les propriétés pour quantity et serving_size
g.add((EX.hasQuantity, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasQuantity, RDFS.domain, EX.Product))
g.add((EX.hasQuantity, RDFS.range, XSD.string))

#g.add((EX.hasServingSize, RDF.type, OWL.DatatypeProperty))
#g.add((EX.hasServingSize, RDFS.domain, EX.Product))
#g.add((EX.hasServingSize, RDFS.range, XSD.string))


g.add((EX.hasPurchasePlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasPurchasePlace, RDFS.domain, EX.Product))
g.add((EX.hasPurchasePlace, RDFS.range, EX.PurchasePlace))

g.add((EX.hasCity, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCity, RDFS.domain, EX.Product))
g.add((EX.hasCity, RDFS.range, EX.City))

"""g.add((EX.hasEcoscore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscore, RDFS.domain, EX.Product))
g.add((EX.hasEcoscore, RDFS.range, EX.Ecoscore))

g.add((EX.hasEcoscoreScore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreScore, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreScore, RDFS.range, EX.EcoscoreScore))

#g.add((EX.hasEcoscoreGrade, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreGrade, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreGrade, RDFS.range, EX.EcoscoreGrade))"""


# Fonction pour encoder les URI de manière sécurisée
def encode_uri(uri):
    return urllib.parse.quote(uri, safe=":/#?&=~")


CACHE_FILE = "country_qcode_cache.json"

def load_cache():
    """
    Charge le cache des résultats depuis un fichier local.
    """
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """
    Sauvegarde le cache des résultats dans un fichier local.
    """
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f)

def normalize_country_name(country_name):
    """
    Nettoie le nom d'un pays pour le rendre standard.
    """
    return country_name.replace("en:", "").split(" - ")[0].strip()

def get_country_qcode(country_name, timeout=10):
    """
    Recherche le Q-code d'un pays donné via l'API SPARQL de Wikidata.
    Inclut des mécanismes de mise en cache et de gestion des erreurs.

    :param country_name: Nom du pays à rechercher
    :param timeout: Durée maximale pour la requête en secondes
    :return: URI complète du pays avec le Q-code, ou une valeur par défaut si une erreur survient
    """
    # Normaliser et nettoyer le nom du pays
    cleaned_country_name = normalize_country_name(country_name)

    # Charger le cache
    cache = load_cache()

    # Vérifier si le résultat est déjà dans le cache
    if cleaned_country_name in cache:
        print(f"Using cached result for {cleaned_country_name}")
        return cache[cleaned_country_name]

    # Configurer le point d'accès SPARQL
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    # Construire la requête SPARQL
    query = f"""
    SELECT ?country WHERE {{
      ?country wdt:P31 wd:Q6256;  # Instance de "pays"
              rdfs:label "{cleaned_country_name}"@en .
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Exécuter la requête
    try:
        sparql.setTimeout(timeout)  # Timeout pour éviter les blocages
        results = sparql.query().convert()
        for result in results.get("results", {}).get("bindings", []):
            qcode_uri = result["country"]["value"]
            # Ajouter le résultat au cache
            cache[cleaned_country_name] = qcode_uri
            save_cache(cache)
            return qcode_uri
    except requests.exceptions.RequestException as req_err:
        print(f"Request error for {country_name}: {req_err}")
    except Exception as e:
        print(f"Error during SPARQL query for {country_name}: {e}")

    # Retourner une valeur par défaut en cas d'erreur
    fallback_uri = "http://www.wikidata.org/entity/Q30"  # Par défaut : Q30 (USA)
    cache[cleaned_country_name] = fallback_uri
    save_cache(cache)
    return fallback_uri


def get_dbpedia_uri(brand_name):
    """Renvoie une URI DBpedia pour une marque donnée."""

    return f"http://dbpedia.org/resource/{brand_name.replace(' ', '_')}"


def get_wikidata_category_uri(category_name, language="en"):
    """
    Recherche le Q-code d'une catégorie donnée sur Wikidata.
    
    Args:
        category_name (str): Le nom de la catégorie à rechercher.
        language (str): La langue de recherche (par défaut "en" pour l'anglais).

    Returns:
        str: L'URI Wikidata pour la catégorie, ou None si elle n'est pas trouvée.
    """
    # URL de l'API Wikidata
    api_url = "https://www.wikidata.org/w/api.php"

    # Paramètres pour rechercher une entité correspondant à la catégorie
    params = {
        "action": "wbsearchentities",
        "search": category_name,
        "language": language,
        "format": "json",
        "type": "item"  # Recherche uniquement les items
    }

    try:
        # Requête à l'API Wikidata
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Lève une exception si le statut HTTP n'est pas 200

        # Récupérer les résultats de la recherche
        data = response.json()
        if data.get("search"):
            # Retourner le premier résultat (Q-code)
            q_code = data["search"][0]["id"]
            return f"http://www.wikidata.org/entity/{q_code}"
        else:
        
            return f"http://www.wikidata.org/entity/Q11070"
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête à l'API Wikidata : {e}")
        return None


# Lire le fichier CSV et ajouter des instances
csv_file_path = "cleaned_file.csv"  # Remplacez par votre fichier CSV

try:
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        print(" debut lecture csv \n")
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info("Début du traitement du fichier CSV.")
        for row in reader:
            if all(col in row for col in important_columns):
                product_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/{row['code']}"))

                # Ajouter le type de produit
                print(" Ajout produit \n")
                g.add((product_uri, RDF.type, DBO.Product))
                g.add((product_uri, SCHEMA.name, Literal(row["product_name"])))
                g.add((product_uri, SCHEMA.url, URIRef(row["url"])))

                # Ajouter la marque avec lien vers DBpedia
                print(" Ajout marque produit, association avec dbpedia \n")
                if row["brands"]:
                    brand_name = row["brands"].strip()
                    brand_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/brand/{row['brands']}"))
                    dbpedia_brand_uri = get_dbpedia_uri(row["brands"])
                    g.add((brand_uri, RDF.type, DBO.Brand))
                    g.add((brand_uri, FOAF.name, Literal(row["brands"])))
                    if dbpedia_brand_uri:
                        
                        g.add((brand_uri, DBO.wikiPage, URIRef(encode_uri(dbpedia_brand_uri))))

                    g.add((product_uri, EX.hasBrand, brand_uri))
                

                # Ajouter des catégories basées sur le champ 'categories' avec lien vers Wikidata
                
                if row["categories"]:
                    categories = row["categories"].split(',')  # Séparer les catégories par des virgules
                    for category in categories:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            category_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/category/{category}"))
                            wikidata_category_uri = get_wikidata_category_uri(category)
                            g.add((category_uri, RDF.type, EX.Category))
                            g.add((category_uri, SCHEMA.name, Literal(category)))
                            g.add((category_uri, DBO.wikiPage, URIRef(encode_uri(wikidata_category_uri))))

                            g.add((product_uri, EX.hasCategory, category_uri))
                     

                # Ajouter des relations d'origine basées sur le champ 'countries' avec lien vers Wikidata
                
                if row["countries"]:
                    countries = row["countries"].split(',')  # Séparer les pays par des virgules
                    for country in countries:
                        country = country.strip()
                        if country:  # Vérifier que le pays n'est pas vide
                            country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/country/{country}"))
                            
                            normalized_country = normalize_country_name(country)
                           
                            qcode_uri = get_country_qcode(normalized_country)
                           
                            #wikidata_country_uri = get_wikidata_uri(country)
                            
                            g.add((country_uri, RDF.type, DBO.Country))
                            
                            g.add((country_uri, FOAF.name, Literal(normalized_country)))
                            
                            
                            g.add((country_uri, DBO.wikiPage, URIRef(encode_uri(qcode_uri))))
                           
                            g.add((product_uri, EX.availableInCountry, country_uri))
             
                if row["allergens"]:
                    allergens = row["allergens"].split(',')  # Séparer les valeurs par des virgules
                    for allergen in allergens:
                        allergen = allergen.strip()
                        if allergen:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                            allergen_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/allergens/{allergen}"))
                            
                            g.add((product_uri, EX.containsAllergen, allergen_uri))
                            g.add((allergen_uri, RDF.type, EX.Allergen))
                            g.add((allergen_uri, SCHEMA.name, Literal(allergen)))
    

                
                # Ajouter des nutriments
                nutrients = [
                    ("energy_100g", "Energy"),
                    ("fat_100g", "Fat"),
                    ("saturated-fat_100g", "Saturated Fat"),
                    ("carbohydrates_100g", "Carbohydrates"),
                    ("sugars_100g", "Sugars"),
                    ("fiber_100g", "Fiber"),
                    ("proteins_100g", "Proteins"),
                    ("salt_100g", "Salt"),
                    ("carbon-footprint_100g", "carbon footprint")
                ]
               
                for nutrient, label in nutrients:
                    if row[nutrient]:
                        nutrient_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/nutrient/{label}"))
                        g.add((product_uri, EX.hasNutrient, nutrient_uri))
                        g.add((nutrient_uri, RDF.type, EX.Nutrient))
                        g.add((nutrient_uri, SCHEMA.name, Literal(label)))
                        g.add((nutrient_uri, SCHEMA.value, Literal(row[nutrient], datatype=XSD.float)))
                  
                        
                # Ajouter des ingrédients
                
                if row["ingredients_text"]:
                    g.add((product_uri, EX.hasIngredientsText, Literal(row["ingredients_text"])))
                 

                # Ajouter les créateurs
                
                if row["creator"]:
                    creator_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/creator/{row['creator']}"))
                    g.add((product_uri, EX.hasCreator, creator_uri))
                    g.add((creator_uri, RDF.type, EX.Creator))
                    g.add((creator_uri, SCHEMA.name, Literal(row["creator"])))
                    

                # Ajouter les lieux de fabrication
               
                if row["manufacturing_places"]:
                    manufacturing = row["manufacturing_places"].split(',')  # Séparer les catégories par des virgules
                    for manu in manufacturing:
                        cmanu = manu.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            manufacturing_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/manufacturing_place/{manu}"))
                            g.add((product_uri, EX.hasManufacturingPlace, manufacturing_place_uri))
                            g.add((manufacturing_place_uri, RDF.type, EX.ManufacturingPlace))
                            g.add((manufacturing_place_uri, SCHEMA.name, Literal(manu)))
                    
                if row["purchase_places"]:
                    purchase_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/purchase_place/{row['purchase_places']}"))
                    g.add((product_uri, EX.hasPurchasePlace, purchase_place_uri))
                    g.add((purchase_place_uri, RDF.type, EX.PurchasePlace))
                    g.add((purchase_place_uri, SCHEMA.name, Literal(row["purchase_places"])))

                # Ajouter les villes
               
                if row["cities"]:
                    city_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/city/{row['cities']}"))
                    g.add((product_uri, EX.hasCity, city_uri))
                    g.add((city_uri, RDF.type, EX.City))
                    g.add((city_uri, SCHEMA.name, Literal(row["cities"])))
                    
                    
                  # Ajouter la quantité (quantity)
                print(" Ajout quantite produit, association avec dbpedia \n")
                if row["quantity"]:
                    g.add((product_uri, EX.hasQuantity, Literal(row["quantity"])))
                   

except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    exit()

# Sauvegarder le graphe OWL
g.serialize(destination=rdf_file_path, format="xml")
print(f"Ontologie OWL générée et sauvegardée dans : {rdf_file_path}")

2024-12-18 15:02:44,387 - INFO - Début du traitement du fichier CSV.


 debut lecture csv 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for Vereinigte Staaten von Amerika
Using cached result for Germany
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for Canada
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for GR
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for CZ
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for France
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout marque produit, association avec dbpedia 

Using cached result for United States
 Ajout quantite produit, association avec dbpedia 

 Ajout produit 

 Ajout 

## Code Python pour interroger un Triplestore

In [ ]:
%pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON

# URL du triplestore (Fuseki)
sparql = SPARQLWrapper("http://localhost:3030/your-dataset-name/query")



# Définir le format de retour (JSON)
sparql.setReturnFormat(JSON)

# Exécuter la requête et obtenir les résultats
results = sparql.query().convert()

In [ ]:
import requests

# Configuration
endpoint = "http://localhost:7200/rest/data/import/upload/test-repository"
files = {'file': open('ontologie_peuplee.ttl', 'rb')}

# Charger le fichier RDF dans GraphDB
response = requests.post(endpoint, files=files)
if response.status_code == 200:
    print("Données chargées avec succès.")
else:
    print(f"Erreur : {response.status_code}")


## Charger les données RDF dans le Triplestore

## Interroger les Données avec SPARQL

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL mise à jour pour vérifier la présence de produits et leurs noms
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?name
WHERE {
  ?product rdf:type dbo:Product .
  ?product schema:name ?name  .
}
LIMIT 10
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, Name: {row['name']}")
else:
    print("Aucun résultat trouvé.")


Aucun résultat trouvé.


In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL mise à jour pour vérifier la présence de produits et leurs noms
#recherche produit par marque
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?brand
WHERE {
  ?product rdf:type dbo:Product .
  ?product ex:hasBrand ?brand  .
}
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, marque: {row['brand']}")
else:
    print("Aucun résultat trouvé.")

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL mise à jour pour vérifier la présence de produits et leurs noms
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?name
WHERE {
  ?product rdf:type dbo:Product .
  ?product schema:name ?name  .
}
LIMIT 10
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, Name: {row['name']}")
else:
    print("Aucun résultat trouvé.")

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.rdf"
g.parse(file_path, format="xml")

# Requête SPARQL mise à jour pour vérifier la présence de produits et leurs noms
#Rechercher les produits avec leurs ingrédients :
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?ingredientsText
WHERE {
  ?product rdf:type dbo:Product .
  ?product ex:hasIngredientsText ?ingredientsText .
}
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, ingredients: {row['ingredientsText']}")
else:
    print("Aucun résultat trouvé.")

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL avec un produit spécifique
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?name ?quantity
WHERE {
  ?product rdf:type dbo:Product .
  ?product schema:name ?name .
  OPTIONAL { ?product ex:hasQuantity ?quantity . }
}

LIMIT 10
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, Name: {row['name']}, quantite: {row['quantity']}, serving_size:{row['serving_size']}")
else:
    print("Aucun résultat trouvé.")

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL avec un produit spécifique
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?name
WHERE {
    ?product rdf:type dbo:Product .
    ?product schema:name ?name .
    FILTER(STR(?product) = "http://produitsalimentaires.fr/product/996")  # Remplacez par un URI de produit réel
}
LIMIT 10
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, Name: {row['name']}")
else:
    print("Aucun résultat trouvé.")

In [ ]:
from rdflib import Graph

# Charger le graphe
g = Graph()
file_path = "output_ontology.owl"
g.parse(file_path, format="xml")

# Requête SPARQL avec un produit spécifique
query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>


SELECT ?product ?name ?quantity ?brand ?category 
WHERE {
    ?product rdf:type dbo:Product ;
             schema:name ?name ;
             ex:hasQuantity ?quantity ;
             ex:hasBrand ?brand ;
             ex:hasCategory ?category .
}
LIMIT 20
"""

# Exécuter la requête et afficher les résultats
results = g.query(query)
if results:
    for row in results:
        print(f"Product: {row['product']}, Name: {row['name']}, quantite: {row['quantity']}, serving_size:{row['serving_size']}, brand: {row['brand']}, category:{row['category']}")
else:
    print("Aucun résultat trouvé.")

## exécuter et visualiser une requête SPARQL Construct

In [ ]:
import rdflib
import networkx as nx
import matplotlib.pyplot as plt

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Effectuer la requête SPARQL CONSTRUCT pour extraire un graphe simplifié
sparql_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

CONSTRUCT {
  ?product rdf:type dbo:Product ;
           schema:name ?productName ;
           ex:hasBrand ?brand ;
           ex:hasCategory ?category ;
           ex:hasCreator ?creator . 
  ?brand a ex:Brand ;
         foaf:name ?brandName .
  ?category a ex:Category ;
            foaf:name ?categoryName .
  
  ?creator a ex:Creator ;
           foaf:name ?creatorName .
}
WHERE {
  # Produit de type Product
  ?product rdf:type dbo:Product ;
           schema:name ?productName .
  OPTIONAL {
    ?product ex:hasBrand ?brand .
    ?brand foaf:name ?brandName .
  }
  
  OPTIONAL {
    ?product ex:hasCategory ?category .
    ?category foaf:name ?categoryName .
  }
  
  OPTIONAL {
    ?product ex:hasCreator ?creator .
    ?creator foaf:name ?creatorName .
  }
}
"""

# Exécuter la requête SPARQL CONSTRUCT
construct_graph = g.query(sparql_query)

# Créer un graphique de type NetworkX à partir du résultat du CONSTRUCT
network_graph = nx.DiGraph()

# Ajouter les triplets du graphe résultant dans le graphique NetworkX
for subj, pred, obj in construct_graph:
    network_graph.add_edge(str(subj), str(obj), label=str(pred))

# Visualisation du graphe avec matplotlib
pos = nx.spring_layout(network_graph, k=0.5, iterations=50)  # Placement des nœuds
plt.figure(figsize=(12, 12))
nx.draw(network_graph, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, font_weight="bold", arrows=True)

# Affichage du graphique
plt.title("Visualisation du Graphe RDF (Résultat de SPARQL CONSTRUCT)")
plt.show()


Ontologie OWL chargée depuis : output_ontology.owl


KeyboardInterrupt: 

## exécuter une requête SPARQL ASK

In [ ]:
import rdflib

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL ASK pour vérifier l'existence d'un produit avec une marque spécifique
sparql_ask_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema1: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

ASK {
  ?product rdf:type dbo:Product ;
           schema1:name ?name ;
           ex:hasBrand ?brand .
  ?brand foaf:name ?brandName .
  
  # Utilisation de LCASE pour rendre la recherche insensible à la casse
  # et LTRIM/RTRIM pour supprimer les espaces inutiles avant et après
  FILTER (LCASE(STR(?name)) = "alfajor dulce de leche" && LCASE(STR(?brandName)) = "la quinta")
}
"""

# Exécuter la requête ASK
result = g.query(sparql_ask_query)

# Afficher le résultat (True ou False)
if result:
    print("La requête ASK a retourné : True")
else:
    print("La requête ASK a retourné : False")


## effectuer la requête SPARQL DESCRIBE et visualiser le graphe

In [ ]:
import rdflib
import networkx as nx
import matplotlib.pyplot as plt

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL DESCRIBE pour obtenir une description d'un produit spécifique
sparql_describe_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema1: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

DESCRIBE ?product
WHERE {
   ?product rdf:type dbo:Product ;
           schema1:name "Alfajor Dulce de leche";
           ex:hasBrand ?brand .
  ?brand foaf:name ?brandName .
}
"""

# Exécuter la requête DESCRIBE
describe_result = g.query(sparql_describe_query)

# Créer un graphique de type NetworkX à partir du résultat du DESCRIBE
network_graph = nx.DiGraph()

# Ajouter les triplets du graphe résultant dans le graphique NetworkX
for subj, pred, obj in describe_result:
    network_graph.add_edge(str(subj), str(obj), label=str(pred))

# Visualisation du graphe avec matplotlib
pos = nx.spring_layout(network_graph, k=0.5, iterations=50)  # Placement des nœuds
plt.figure(figsize=(12, 12))
nx.draw(network_graph, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, font_weight="bold", arrows=True)

# Affichage du graphique
plt.title("Visualisation du Graphe RDF (Résultat de SPARQL DESCRIBE)")
plt.show()

## exécuter la requête SPARQL SELECT avec FILTER

In [ ]:
import rdflib

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL SELECT avec FILTER
sparql_select_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?product ?name ?quantity ?categoryName
WHERE {
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasCategory ?category ;
           ex:hasQuantity ?quantity .
  ?category foaf:name ?categoryName .
}
LIMIT 10
"""

# Exécuter la requête SELECT
results = g.query(sparql_select_query)

# Afficher les résultats
for row in results:
    print(f"Produit: {row['product']}, Nom: {row['name']}, Prix: {row['price']}, Catégorie: {row['category']}")

In [ ]:
import rdflib

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL SELECT avec FILTER
sparql_select_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?product ?name ?quantity ?categoryName
WHERE {
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasCategory ?category ;
           ex:hasQuantity ?quantity .
  
  # Extraire la partie numérique de quantity en utilisant REGEX
  BIND (xsd:decimal(REPLACE(?quantity, "[^0-9\\.]", "")) AS ?numericQuantity)

  # Appliquer un filtre pour les produits dont la quantité est supérieure à 10
  FILTER (?numericQuantity > 10)
  
  ?category foaf:name ?categoryName .
}
LIMIT 10
"""

# Exécuter la requête SELECT
results = g.query(sparql_select_query)

# Afficher les résultats
for row in results:
    print(f"Produit: {row['product']}, Nom: {row['name']}, Prix: {row['price']}, Catégorie: {row['category']}")

## requete complexe

In [ ]:
import rdflib

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

"""La requête CONSTRUCT génère un graphe RDF avec des informations détaillées sur les produits, 
leurs marques et catégories, tout en utilisant des filtres et des extractions sur la quantité pour obtenir des données plus structurées. """
# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL SELECT avec FILTER
sparql_select_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

CONSTRUCT {
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasQuantity ?quantity ;
           ex:hasCategory ?category ;
           ex:hasBrand ?brand .
  
  ?brand foaf:name ?brandName .
  ?category foaf:name ?categoryName .
  
  # Si vous avez besoin de spécifier la quantité en tant que valeur numérique
  ?product ex:quantityValue ?numericQuantity .
}
WHERE {
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasCategory ?category ;
           ex:hasQuantity ?quantity ;
           ex:hasBrand ?brand .
  
  # Extraire la partie numérique de quantity en utilisant REGEX
  BIND (xsd:decimal(REPLACE(?quantity, "[^0-9\\.]", "")) AS ?numericQuantity)
  
  # Associer chaque produit à une marque et une catégorie
  ?brand foaf:name ?brandName .
  ?category foaf:name ?categoryName .

  # Filtrer les produits dont la quantité est supérieure à 10 et qui appartiennent à la catégorie "Dulces de Navidad"
  FILTER (?numericQuantity > 10 && LCASE(STR(?categoryName)) = "dulces de navidad")
}
LIMIT 20
"""

# Exécuter la requête SELECT
results = g.query(sparql_select_query)

# Afficher les résultats
for row in results:
    print(f"Produit: {row['product']}, Nom: {row['name']}, Prix: {row['price']}, Catégorie: {row['category']}")

In [ ]:
import rdflib

# Charger le graphe RDF avec rdflib
g = rdflib.Graph()

"""Cette requête complexe fait le lien entre les produits, 
leur marque, leur catégorie et leur quantité, tout en utilisant des fonctions 
comme REPLACE pour extraire la partie numérique de quantity et en appliquant 
des filtres pour ne sélectionner que les produits répondant aux critères définis."""
# Charger le fichier RDF/XML existant
g.parse("output_ontology.rdf", format="xml")

# Requête SPARQL SELECT avec FILTER
sparql_select_query = """
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?product ?name ?quantity ?numericQuantity ?brandName ?categoryName
WHERE {
  # Sélectionner les produits de type dbo:Product avec un nom spécifique
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasCategory ?category ;
           ex:hasQuantity ?quantity ;
           ex:hasBrand ?brand .

  # Extraire la partie numérique de quantity en utilisant REGEX et la convertir en nombre
  BIND (xsd:decimal(REPLACE(?quantity, "[^0-9\\.]", "")) AS ?numericQuantity)

  # Associer chaque produit à une marque et une catégorie
  ?brand foaf:name ?brandName .
  ?category foaf:name ?categoryName .

  # Filtrer les produits dont la quantité est supérieure à 10 et qui appartiennent à la catégorie 'Dulces de Navidad'
  FILTER (?numericQuantity > 10 && LCASE(STR(?categoryName)) = "dulces de navidad"
}

ORDER BY ?name
"""

# Exécuter la requête SELECT
results = g.query(sparql_select_query)

# Afficher les résultats
for row in results:
    print(f"Produit: {row['product']}, Nom: {row['name']}, Prix: {row['price']}, Catégorie: {row['category']}")

## Requêtes SPARQL Fédérées

### Récupérer les produits, leurs marques et les informations depuis DBpedia

In [ ]:
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?product ?name ?category ?quantity ?brand ?brandName ?brandWikiPage ?country ?countryName ?countryWikiPage
WHERE {
  # Données locales
  ?product rdf:type dbo:Product ;
           schema:name ?name ;
           ex:hasCategory ?category ;
           ex:hasQuantity ?quantity ;
           ex:hasBrand ?brand ;
           ex:availableInCountry ?country .

  # Obtenir le nom de la marque
  ?brand foaf:name ?brandName .

  # Lier la marque à DBpedia pour obtenir des informations supplémentaires
  OPTIONAL {
    SERVICE <https://dbpedia.org/sparql> {
      ?brand dbo:wikiPage ?brandWikiPage .
    }
  }

  # Obtenir le nom du pays
  ?country foaf:name ?countryName .

  # Lier le pays à Wikidata pour obtenir la page du pays
  OPTIONAL {
    SERVICE <https://query.wikidata.org/sparql> {
      ?country rdfs:seeAlso ?countryWikiPage .
    }
  }
}
LIMIT 100


### Récupérer des informations sur les produits d'une catégorie et enrichir avec Wikidata

In [ ]:
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX wikidata: <http://www.wikidata.org/ontology#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?product ?name ?category ?categoryWikiPage
WHERE {
  # Interroger les produits de la catégorie "Beverages"
  ?product a ex:Product ;
           ex:name ?name ;
           ex:hasCategory ?category .

  # Lier la catégorie à Wikidata pour obtenir la page Wiki de la catégorie
  SERVICE <https://query.wikidata.org/sparql> {
    ?category wikidata:category ?categoryWikiPage .
  }
}


In [ ]:
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX schema: <http://schema.org/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

CONSTRUCT {
  ?product a dbo:Product ;
           schema:name ?productName ;
           ex:hasBrand ?brand ;
           ex:hasCategory ?category ;
           ex:containsAllergen ?allergen ;
           ex:availableInCountry ?country .

  ?brand a dbo:Brand ;
         foaf:name ?brandName ;
         schema:identifier ?brandWiki .

  ?category a dbo:Category ;
            foaf:name ?categoryName ;
            schema:identifier ?categoryWiki .
}
WHERE {
  # Récupérer les informations du produit local
  ?product a dbo:Product ;
           schema:name ?productName ;
           ex:hasBrand ?brand ;
           ex:hasCategory ?category ;
           ex:containsAllergen ?allergen ;
           ex:availableInCountry ?country .

  # Récupérer les informations de la marque locale
  ?brand a dbo:Brand ;
         foaf:name ?brandName .

  # Lier la marque à Wikidata pour enrichissement
  SERVICE <https://query.wikidata.org/sparql> {
    OPTIONAL {
      ?brandWiki foaf:name ?brandName .
      ?brandWiki wdt:P31 wd:Q167169 .
    }
  }

  # Récupérer les informations de la catégorie locale
  ?category a dbo:Category ;
            foaf:name ?categoryName .

  # Lier la catégorie à Wikidata pour enrichissement
  SERVICE <https://query.wikidata.org/sparql> {
    OPTIONAL {
      ?categoryWiki foaf:name ?categoryName .
      ?categoryWiki wdt:P31 wd:Q12308941 .
    }
  }
}


### Récupérer des produits avec un prix supérieur à 10 et enrichir avec des informations depuis DBpedia et Wikidata

In [ ]:
PREFIX ex: <http://produitsalimentaires.fr/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX wikidata: <http://www.wikidata.org/ontology#>

SELECT ?product ?name ?price ?brand ?brandWikiPage ?country ?countryWikiPage
WHERE {
  # Interroger les produits avec un prix supérieur à 10
  ?product a ex:Product ;
           ex:name ?name ;
           ex:price ?price ;
           ex:hasBrand ?brand ;
           ex:availableInCountry ?country .
  FILTER (?price > 10)

  # Lier la marque à DBpedia pour obtenir des informations supplémentaires
  SERVICE <http://dbpedia.org/sparql> {
    ?brand dbo:wikiPage ?brandWikiPage .
  }

  # Lier le pays à Wikidata pour obtenir la page du pays
  SERVICE <https://query.wikidata.org/sparql> {
    ?country wikidata:country ?countryWikiPage .
  }
}


In [ ]:
import rdflib
from rdflib import Graph
import networkx as nx
import matplotlib.pyplot as plt

# Charger l'ontologie depuis un fichier OWL
file_path = "output_ontology.owl"

# Charger le fichier OWL
g = Graph()
try:
    g.parse(file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()

# Créer un graphe NetworkX
G = nx.Graph()

# Parcourir les triples pour ajouter des nœuds et des arêtes
for subj, pred, obj in g:
    # Ajouter des nœuds pour les sujets et les objets
    G.add_node(str(subj))
    G.add_node(str(obj))
    # Ajouter une arête entre les sujets et les objets
    G.add_edge(str(subj), str(obj), label=str(pred))

# Visualisation avec matplotlib
fig, ax = plt.subplots(figsize=(15, 10))
pos = nx.spring_layout(G)  # Positionnement des nœuds
nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, ax=ax)
labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, ax=ax)

plt.title("Visualisation de l'ontologie")
# Sauvegarder l'image du graphe
plt.savefig("ontology_graph2.png", format="png")
plt.show()